### Uc_Merged dataseti

In [1]:
data_name = 'uc_merced'

#### Tensorflow datasetlaridan uc_merged datasetini yuklab olamiz

In [2]:
import tensorflow_datasets as tfds

dataset, info = tfds.load(
    data_name,
    split=['train'],
    data_dir="dataset\\",
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
    batch_size=-1
)

In [3]:
info

tfds.core.DatasetInfo(
    name='uc_merced',
    full_name='uc_merced/2.0.0',
    description="""
    UC Merced is a 21 class land use remote sensing image dataset, with 100 images
    per class. The images were manually extracted from large images from the USGS
    National Map Urban Area Imagery collection for various urban areas around the
    country. The pixel resolution of this public domain imagery is 0.3 m.
    
    While most images are 256x256 pixels, there are 44 images with different shape.
    """,
    homepage='http://weegee.vision.ucmerced.edu/datasets/landuse.html',
    data_path='dataset\\uc_merced\\2.0.0',
    file_format=tfrecord,
    download_size=317.07 MiB,
    dataset_size=238.63 MiB,
    features=FeaturesDict({
        'filename': Text(shape=(), dtype=string),
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=21),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,


#### Image va labellarni preprocessingdan o'tkazamiz 

In [4]:
images,labels = tfds.as_numpy(dataset[0])

In [5]:
import cv2
import numpy as np

def preprocessing(images,label):
    images_ = []
    
    for i in range(len(images)):
        image = cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB)
        image = cv2.resize(image,(224,224),interpolation = cv2.INTER_AREA)
        image = image/127.5-1.0
        image = image.astype('float16')
        
        images_.append(image)
        
    label = label.astype('uint16')
    images = np.array(images_)
    return images,label

In [6]:
images,labels = preprocessing(images,labels)

In [7]:
images[0].shape

(224, 224, 3)

#### Datasetni train va test datalarga ajratamiz

In [8]:
import tensorflow as tf
train_ds = tf.data.Dataset.from_tensor_slices((images[:2000], labels[:2000])).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((images[2000:], labels[2000:])).batch(32)

### Model o'qitish uchun VGG-19 arxitekturasidan foydalanamiz
#### 1. VGG-19 transfer learning yordamida o'qitish

In [9]:
vgg19 = tf.keras.applications.VGG19(input_shape=(224,224,3), include_top=False) # 

#####  weight va biaslarni Non-trainable holatga o'tkazamiz

In [10]:
vgg19.trainable=False

##### Arxitekturaning Convolutional qismi

In [11]:
vgg19.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

##### Modelni ANN qismini datasetimizga moslab qurib olamiz

In [12]:
model = tf.keras.Sequential([
    vgg19,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2048,activation = tf.keras.activations.relu),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1024,activation = tf.keras.activations.relu),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512,activation = tf.keras.activations.relu),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(256,activation = tf.keras.activations.relu),
    tf.keras.layers.Dense(21,activation = tf.keras.activations.softmax),
])

##### Fully-connected 

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 2048)              51382272  
                                                                 
 batch_normalization (BatchN  (None, 2048)             8192      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense_1 (Dense)             (None, 1024)              2098176   
                                                        

##### callbacks parametrlarini sozlab olamiz

In [14]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_sparse_categorical_accuracy', 
    patience=3)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'models\VGG-19\model_with_transfer.h5', 
     monitor='val_sparse_categorical_accuracy', mode='max', 
     save_best_only=True, 
     verbose=1)

log_directory = "logs"
tensorboard = tf.keras.callbacks.TensorBoard(
    log_dir=log_directory,
    histogram_freq=0,
    write_graph=True,
    write_images=False,
    write_steps_per_second=False,
    update_freq="epoch",
    profile_batch=2,
    embeddings_freq=0,
    embeddings_metadata=None
)

##### Modelni compile va fit qilamiz

In [15]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics = tf.keras.metrics.SparseCategoricalAccuracy())

history = model.fit(train_ds, epochs = 5, validation_data = test_ds, callbacks = [early_stopping, checkpoint, tensorboard])

Epoch 1/5
63/63 [==============================] - ETA: 0s - loss: 1.0229 - sparse_categorical_accuracy: 0.6815
Epoch 1: val_sparse_categorical_accuracy improved from -inf to 0.37000, saving model to models\VGG-19\model_with_transfer.h5
63/63 [==============================] - 260s 4s/step - loss: 1.0229 - sparse_categorical_accuracy: 0.6815 - val_loss: 8.4592 - val_sparse_categorical_accuracy: 0.3700
Epoch 2/5
63/63 [==============================] - ETA: 0s - loss: 0.2314 - sparse_categorical_accuracy: 0.9170
Epoch 2: val_sparse_categorical_accuracy improved from 0.37000 to 0.52000, saving model to models\VGG-19\model_with_transfer.h5
63/63 [==============================] - 261s 4s/step - loss: 0.2314 - sparse_categorical_accuracy: 0.9170 - val_loss: 3.7798 - val_sparse_categorical_accuracy: 0.5200
Epoch 3/5
63/63 [==============================] - ETA: 0s - loss: 0.1142 - sparse_categorical_accuracy: 0.9685
Epoch 3: val_sparse_categorical_accuracy improved from 0.52000 to 0.84000, 

#### 2. VGG-19 arxitekturasini noldan o'zimiz quramiz 

In [16]:
class MyModel(tf.keras.models.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        
        self.block1_1conv = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.relu, padding="same")
        self.block1_2conv = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.relu, padding="same")
        
        self.block2_1conv = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.relu, padding="same")
        self.block2_2conv = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.relu, padding="same")
        
        self.block3_1conv = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.relu, padding="same")
        self.block3_2conv = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.relu, padding="same")
        self.block3_3conv = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.relu, padding="same")
        self.block3_4conv = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.relu, padding="same")
        
        self.block4_1conv = tf.keras.layers.Conv2D(512, (3, 3), activation=tf.keras.activations.relu, padding="same")
        self.block4_2conv = tf.keras.layers.Conv2D(512, (3, 3), activation=tf.keras.activations.relu, padding="same")
        self.block4_3conv = tf.keras.layers.Conv2D(512, (3, 3), activation=tf.keras.activations.relu, padding="same")
        self.block4_4conv = tf.keras.layers.Conv2D(512, (3, 3), activation=tf.keras.activations.relu, padding="same")
        
        self.block5_1conv = tf.keras.layers.Conv2D(512, (3, 3), activation=tf.keras.activations.relu, padding="same")
        self.block5_2conv = tf.keras.layers.Conv2D(512, (3, 3), activation=tf.keras.activations.relu, padding="same")
        self.block5_3conv = tf.keras.layers.Conv2D(512, (3, 3), activation=tf.keras.activations.relu, padding="same")
        self.block5_4conv = tf.keras.layers.Conv2D(512, (3, 3), activation=tf.keras.activations.relu, padding="same")
        
        self.pool = tf.keras.layers.MaxPooling2D((2, 2),strides=2)
        
        self.flatten = tf.keras.layers.Flatten()
        self.hidden_layer1 = tf.keras.layers.Dense(2048, activation=tf.keras.activations.relu)
        self.hidden_layer2 = tf.keras.layers.Dense(1024, activation=tf.keras.activations.relu)
        self.hidden_layer3 = tf.keras.layers.Dense(512, activation=tf.keras.activations.relu)
        self.hidden_layer4 = tf.keras.layers.Dense(256, activation=tf.keras.activations.relu)        
        self.output_layer = tf.keras.layers.Dense(21, activation=tf.keras.activations.softmax)
        
        
    def call(self, x):
        '''Convolutional Layer'''
        # Block 1
        conv1 = self.block1_1conv(x) # 224x224x64
        conv2 = self.block1_2conv(conv1) # 224x224x64
        pooling = self.pool(conv2) # 112x112x64

        # Block 2
        conv3 = self.block2_1conv(pooling) # 112x112x128
        conv4 = self.block2_2conv(conv3) # 112x112x128
        pooling = self.pool(conv4) # 56x56x128

        # Block 3
        conv5 = self.block3_1conv(pooling) # 56x56x256
        conv6 = self.block3_2conv(conv5) # 56x56x256
        conv7 = self.block3_3conv(conv6) # 56x56x256
        conv8 = self.block3_4conv(conv7) # 56x56x256
        pooling = self.pool(conv8) # 28x28x256
        
        # Block 4
        conv9  = self.block4_1conv(pooling) # 28x28x512
        conv10 = self.block4_2conv(conv9) # 28x28x512
        conv11 = self.block4_3conv(conv10) # 28x28x512
        conv12 = self.block4_4conv(conv11) # 28x28x512
        pooling = self.pool(conv12) # 14x14x512
        
        # Block 5
        conv13 = self.block5_1conv(pooling) # 14x14x512
        conv14 = self.block5_2conv(conv13) # 14x14x512
        conv15 = self.block5_3conv(conv14) # 14x14x512
        conv16 = self.block5_4conv(conv15) # 14x14x512
        pooling = self.pool(conv16) # 7x7x512
        
        '''Classification Layer'''
        flatten = self.flatten(pooling)
        
        hidden1 = self.hidden_layer1(flatten)
        hidden2 = self.hidden_layer2(hidden1)
        hidden3 = self.hidden_layer3(hidden2)
        hidden4 = self.hidden_layer4(hidden3)
        
        return self.output_layer(hidden4)

model = MyModel()

In [17]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [18]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [19]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:

        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [20]:
@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [21]:
EPOCHS = 5

for epoch in range(EPOCHS):

    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(f"** {epoch + 1}-epoch:\nloss: {train_loss.result()}\tsparse_categorical_accuracy: {train_accuracy.result()}\nval_loss: {test_loss.result()}\tval_sparse_categorical_accuracy: {test_accuracy.result()}")

** 1-epoch:
loss: 3.0516700744628906	sparse_categorical_accuracy: 0.039000000804662704
val_loss: 3.046691417694092	val_sparse_categorical_accuracy: 0.03999999910593033
** 2-epoch:
loss: 3.0452542304992676	sparse_categorical_accuracy: 0.03700000047683716
val_loss: 3.046025276184082	val_sparse_categorical_accuracy: 0.03999999910593033
** 3-epoch:
loss: 3.044947624206543	sparse_categorical_accuracy: 0.03700000047683716
val_loss: 3.0476186275482178	val_sparse_categorical_accuracy: 0.029999999329447746
** 4-epoch:
loss: 3.0448215007781982	sparse_categorical_accuracy: 0.04149999842047691
val_loss: 3.0484046936035156	val_sparse_categorical_accuracy: 0.029999999329447746
** 5-epoch:
loss: 3.044779062271118	sparse_categorical_accuracy: 0.04100000113248825
val_loss: 3.049091339111328	val_sparse_categorical_accuracy: 0.029999999329447746


In [22]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  1792      
                                                                 
 conv2d_1 (Conv2D)           multiple                  36928     
                                                                 
 conv2d_2 (Conv2D)           multiple                  73856     
                                                                 
 conv2d_3 (Conv2D)           multiple                  147584    
                                                                 
 conv2d_4 (Conv2D)           multiple                  295168    
                                                                 
 conv2d_5 (Conv2D)           multiple                  590080    
                                                                 
 conv2d_6 (Conv2D)           multiple                  590

In [23]:
model.save('models/VGG-19/model_without_transfer')

INFO:tensorflow:Assets written to: models/VGG-19/model_without_transfer\assets


INFO:tensorflow:Assets written to: models/VGG-19/model_without_transfer\assets
